## 1- importer les bibliothéques

In [2]:
import os
from PIL import Image
import numpy as np 
import pandas as pd
from skimage.io import imread
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import make_scorer, recall_score, precision_score
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation, BatchNormalization
from keras.backend import dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.optimizers import Adam
from sklearn.metrics import classification_report


## 2- fixer les path des données augmenter 

In [3]:
sj_fliped='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_fliped'
sj_zoom='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_zoom'
sj_rotation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_rotation'
sj_translation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_translation'
sj_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_distortion'
sj_rot_trans_flip='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_fliped_rotation_translation'
sj_flip_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_fliped_distortion'
sj_rot_tran='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_rotation_translation'

nsj_fliped='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_fliped'
nsj_zoom='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_zoom'
nsj_rotation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_rotation'
nsj_translation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_translation'
nsj_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_distortion'
nsj_rot_trans_flip='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_fliped_rotation_translation'
nsj_flip_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_fliped_distortion'
nsj_rot_tran='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_rotation_translation'

In [4]:
sj_path='/kaggle/input/sjdata/DATA_(512,512)/sj_data'
nsj_path='/kaggle/input/sjdata/DATA_(512,512)/nsj_data'

## Mise en Forme des images

In [5]:

def mise_tenseur (path, nl, nc,pl):
    images_liste = os.listdir(path)
    images_liste.sort()
    Nb_images = len(images_liste)
    tenseur = np.empty((Nb_images, nl, nc, pl), dtype=np.uint8)
    k = 0
    for im in images_liste:
       
        img = imread(path+'/'+im)
        tenseur[k, :, :, 0] = img
        k +=1
    return(tenseur)

In [6]:
# diviser la base en 2
label_nsj = [0] * len(mise_tenseur(nsj_path,512,512,1))  # labels pour la première classe (0)
label_sj = [1] *len(mise_tenseur(sj_path,512,512,1)) # labels pour la deuxième classe (1)
# Fusionner les données et les labels en un seul tableau multidimensionnel
data2 = np.concatenate([mise_tenseur(nsj_path,512,512,1), mise_tenseur(sj_path,512,512,1)])
print(len(data2))
labels2 = np.concatenate([label_nsj, label_sj])
len(labels2)




225


225

In [7]:
sj_data = []
sj_data.extend(mise_tenseur(sj_rotation,512,512,1))
sj_data.extend(mise_tenseur(sj_fliped, 512,512,1))
sj_data.extend(mise_tenseur(sj_zoom,512,512,1))
sj_data.extend(mise_tenseur(sj_translation,512,512,1))
sj_data.extend(mise_tenseur(sj_distortion,512,512,1))
sj_data.extend(mise_tenseur(sj_rot_trans_flip,512,512,1))
sj_data.extend(mise_tenseur(sj_flip_distortion,512,512,1))

In [8]:
nsj_data = []
nsj_data.extend(mise_tenseur(nsj_zoom,512,512,1))
nsj_data.extend(mise_tenseur(nsj_fliped,512,512,1))
nsj_data.extend(mise_tenseur(nsj_translation,512,512,1))
nsj_data.extend(mise_tenseur(nsj_distortion,512,512,1))
nsj_data.extend(mise_tenseur(nsj_rot_trans_flip,512,512,1))
nsj_data.extend(mise_tenseur(nsj_flip_distortion,512,512,1))
nsj_data.extend(mise_tenseur(nsj_rotation,512,512,1))


In [9]:
len(nsj_data)

1650

In [10]:
labels1 = [0] * len(nsj_data)  # labels pour la première classe (0)
labels2 = [1] * len(sj_data)  # labels pour la deuxième classe (1)

# Fusionner les données et les labels en un seul tableau multidimensionnel
data = np.concatenate([nsj_data, sj_data])
print(len(data))
labels = np.concatenate([labels1, labels2])
len(labels)


3375


3375

# cross validation

In [11]:
# définir la fonction pour créer le modèle
def create_model():
    model3 = Sequential()
    model3.add(Conv2D(32,(11,11),activation='relu', input_shape = (512,512,1)))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(32, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(32, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(16, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(16, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Flatten())
    model3.add(Dense(128,activation='relu'))
    model3.add(Dense(64,activation='relu'))
    model3.add(Dense(16, activation='relu'))
    model3.add(Dense(1,activation='sigmoid'))
    model3.compile(optimizer='adam',
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
    return model3


# Définition des listes pour stocker les mesures de performance pour chaque fold
precisions = []
recalls = []
f1_scores = []
lr_plus = []
lr_minus = []

# Définition de la cross-validation
kfold = KFold(n_splits=7, shuffle=True, random_state=42)

# Boucle sur les folds
for fold, (train, test) in enumerate(kfold.split(data, labels)):
    # Création du modèle
    model = create_model()
    # Entraînement du modèle sur les données d'entraînement du fold
    model.fit(data[train], labels[train], epochs=30, batch_size=32, verbose=0)
    # Affichage des résultats
    score = model.evaluate(data[test], labels[test], verbose=0)
    # Évaluation du modèle sur les données de test du fold
    y_pred = model.predict(data[test])
    y_pred_classes = np.where(y_pred>0.5,1.,0)
 # Calcul des mesures de performance
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels[test], y_pred_classes, average='binary')
    tn, fp, fn, tp = confusion_matrix(labels[test], y_pred_classes).ravel()
    lr_plus_fold = recall / (1 - tn / (tn + fp))
    lr_minus_fold = (1 - recall) / (tn / (tn + fp))
    # Stockage des mesures de performance pour le fold actuel
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)
    lr_plus.append(lr_plus_fold)
    lr_minus.append(lr_minus_fold)
    # Affichage des résultats
    print('Fold', fold+1, '- Loss:', score[0], '- Accuracy:', score[1])
    print('Precision:', precision, '- Recall:', recall, '- F1 score:', f1_score)
    print('LR+:', lr_plus, '- LR-:', lr_minus)

16/16 [==============================] - 1s 39ms/step
Fold 1 - Loss: 0.10443076491355896 - Accuracy: 0.979296088218689
Precision: 0.9823008849557522 - Recall: 0.9736842105263158 - F1 score: 0.9779735682819383
LR+: [62.07236842105242] - LR-: [0.02673516460473891]
16/16 [==============================] - 1s 38ms/step
Fold 2 - Loss: 0.1349663883447647 - Accuracy: 0.9751037359237671
Precision: 0.9803149606299213 - Recall: 0.97265625 - F1 score: 0.9764705882352941
LR+: [62.07236842105242, 43.96406249999993] - LR-: [0.02673516460473891, 0.027962386877828056]
16/16 [==============================] - 1s 38ms/step
Fold 3 - Loss: 0.047464363276958466 - Accuracy: 0.9937759041786194
Precision: 0.9916666666666667 - Recall: 0.99581589958159 - F1 score: 0.9937369519832986
LR+: [62.07236842105242, 43.96406249999993, 120.99163179916374] - LR-: [0.02673516460473891, 0.027962386877828056, 0.004218823243459043]
16/16 [==============================] - 1s 38ms/step
Fold 4 - Loss: 0.0040503619238734245 - Ac

/tmp/ipykernel_31/1574499261.py:49: RuntimeWarning: divide by zero encountered in double_scalars
  lr_plus_fold = recall / (1 - tn / (tn + fp))


16/16 [==============================] - 1s 38ms/step
Fold 5 - Loss: 0.04065448045730591 - Accuracy: 0.9917012453079224
Precision: 0.9885931558935361 - Recall: 0.9961685823754789 - F1 score: 0.9923664122137406
LR+: [62.07236842105242, 43.96406249999993, 120.99163179916374, inf, 73.38441890166054] - LR-: [0.02673516460473891, 0.027962386877828056, 0.004218823243459043, 0.0038910505836575737, 0.0038841435551337806]
16/16 [==============================] - 1s 38ms/step
Fold 6 - Loss: 0.6952910423278809 - Accuracy: 0.4771784245967865
Precision: 0.47717842323651455 - Recall: 1.0 - F1 score: 0.6460674157303371
LR+: [62.07236842105242, 43.96406249999993, 120.99163179916374, inf, 73.38441890166054, 1.0] - LR-: [0.02673516460473891, 0.027962386877828056, 0.004218823243459043, 0.0038910505836575737, 0.0038841435551337806, nan]


/tmp/ipykernel_31/1574499261.py:50: RuntimeWarning: invalid value encountered in double_scalars
  lr_minus_fold = (1 - recall) / (tn / (tn + fp))


16/16 [==============================] - 1s 38ms/step
Fold 7 - Loss: 0.19687478244304657 - Accuracy: 0.9253112077713013
Precision: 0.8978102189781022 - Recall: 0.968503937007874 - F1 score: 0.9318181818181818
LR+: [62.07236842105242, 43.96406249999993, 120.99163179916374, inf, 73.38441890166054, 1.0, 7.886389201349828] - LR-: [0.02673516460473891, 0.027962386877828056, 0.004218823243459043, 0.0038910505836575737, 0.0038841435551337806, nan, 0.03590551181102369]


In [12]:
# Affichage des résultats
print('Precision : %.3f' % np.mean(precisions))
print('Recall : %.3f' % np.mean(recalls))
print('F1-score : %.3f' % np.mean(f1_scores))
print('LR+ :  ',lr_plus)
print('LR- : %.3f' % np.mean(lr_minus))

Precision : 0.903
Recall : 0.986
F1-score : 0.931
LR+ :   [62.07236842105242, 43.96406249999993, 120.99163179916374, inf, 73.38441890166054, 1.0, 7.886389201349828]
LR- : nan
